# BayesDB analysis of fish bouts in relation to paramecia characteristics

## 1. Ingesting the data

### a. Launching BayesDB

In [1]:
from csv_analysis import invert_all_bouts
import pandas as pd
#drct = '042318_6/'
drct = '091418_bdb/'
csv_file = drct + 'all_huntbouts.csv'
data = pd.read_csv(csv_file)
#invert_all_bouts(data, drct)


In [2]:
%load_ext jupyter_probcomp.magics

session_id: jovyan@nightcrawler2-notebook_2019-01-08T03:39:17.791744_2


/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
%matplotlib inline
%vizgpm inline

<IPython.core.display.Javascript object>

In [4]:
#!rm -f 091418_bdb/bdb_hunts_inverted.bdb
#%bayesdb -j 091418_bdb/bdb_hunts_inverted.bdb

!rm -f 091418_bdb/bdb_hunts.bdb
%bayesdb -j 091418_bdb/bdb_hunts.bdb


u'Loaded: 091418_bdb/bdb_hunts.bdb'

### b. Loading and printing the data from a `.csv` file

In [5]:
%sql DROP TABLE IF EXISTS bout_table;
#%bql CREATE TABLE bout_table FROM '091418_bdb/huntbouts_inverted.csv' 
%bql CREATE TABLE bout_table FROM '091418_bdb/all_huntbouts.csv' 

""


In [6]:
%sql SELECT * FROM bout_table LIMIT 25;

,Bout Number,Bout Az,Bout Alt,Bout Dist,Bout Delta Pitch,Bout Delta Yaw,Para Az,Para Alt,Para Dist,Para Az Velocity,Para Alt Velocity,Para Dist Velocity,Para Az Accel,Para Alt Accel,Para Dist Accel,Postbout Para Az,Postbout Para Alt,Postbout Para Dist,Strike Or Abort
0,0.0,-1.517935,0.209603,79.683312,0.154864,-0.597033,-0.977032,0.629180,354.291281,-0.018158,0.248046,63.812992,0.000281,0.010861,-0.604383,-0.175346,0.592776,299.960989,2.0
1,1.0,-0.091744,-0.041809,36.699616,-0.003720,-0.025881,-0.162832,0.654781,305.528267,-0.167410,-0.006146,59.091504,0.157992,0.034684,18.509784,-0.158165,0.736650,286.473525,2.0
2,2.0,-0.696157,1.136643,99.484831,0.394846,-0.325807,-0.179093,0.818221,281.527199,-0.009887,-0.040281,40.084665,-0.189265,0.072411,-12.334977,0.171628,0.271518,200.860390,2.0
3,3.0,0.076014,0.096790,44.498897,0.006143,-0.006103,0.073963,0.210280,134.098036,-0.201641,-0.130749,28.201272,0.054981,-0.077747,2.364815,0.012218,0.210299,95.292186,2.0
4,4.0,-0.091702,-0.448835,37.549377,-0.230556,-0.013962,0.108441,0.299536,77.905007,-1.066720,-0.747382,56.823983,0.116278,0.019021,24.947333,0.221962,0.746375,60.931606,2.0
5,0.0,-0.765298,0.134860,46.688688,0.016697,-0.213122,-0.442195,0.487576,78.876586,0.516194,-1.271139,91.357699,-0.045214,0.153233,6.152110,0.053742,0.512896,63.664664,1.0
6,0.0,-0.824940,-0.231436,31.337199,-0.035583,-0.203822,-0.209214,0.380260,483.832747,0.275126,-0.092516,43.492538,-0.005240,0.004291,16.188819,0.095739,0.439365,475.184808,3.0
7,1.0,-0.186528,0.020155,28.613087,-0.039567,-0.083962,0.149280,0.407985,486.236400,0.326126,-0.148263,84.071759,0.032529,0.031554,-22.180793,0.336323,0.441544,474.126460,3.0
8,2.0,0.536859,0.085750,73.631208,0.329754,0.269262,0.372042,0.442071,482.865043,0.326013,-0.035337,121.827692,-0.031328,-0.004561,15.843336,0.130932,0.193241,439.046647,3.0
9,3.0,0.376981,-0.223299,85.855307,-0.243187,0.195775,0.145289,0.155442,443.265145,0.242259,0.003376,92.204553,-0.044424,0.036711,19.591807,-0.018229,0.441414,395.166021,3.0


In [7]:
%bql .nullify bout_table ''
%bql .nullify bout_table 'nan'

Nullified 0 cells
Nullified 87 cells


## 2. Automatically learning a CrossCat probabilistic model

### a. Defining an analysis population

In [8]:
%bql GUESS SCHEMA FOR bout_table

,column,stattype,num_distinct,reason
0,Bout Number,numerical,25.0,There are at least 20 unique numerical values...
1,Bout Az,numerical,818.0,There are at least 20 unique numerical values...
2,Bout Alt,numerical,818.0,There are at least 20 unique numerical values...
3,Bout Dist,numerical,818.0,There are at least 20 unique numerical values...
4,Bout Delta Pitch,numerical,813.0,There are at least 20 unique numerical values...
5,Bout Delta Yaw,numerical,818.0,There are at least 20 unique numerical values...
6,Para Az,numerical,818.0,There are at least 20 unique numerical values...
7,Para Alt,numerical,818.0,There are at least 20 unique numerical values...
8,Para Dist,numerical,818.0,There are at least 20 unique numerical values...
9,Para Az Velocity,numerical,811.0,There are at least 20 unique numerical values...


In [9]:
%%mml
DROP POPULATION IF EXISTS bout_population;
CREATE POPULATION bout_population FOR bout_table WITH SCHEMA (GUESS STATTYPES OF (*);
                          IGNORE "Bout Number";)

""


### b. Creating and analyzing a probabilistic model (automatically)

In [10]:
%mml CREATE GENERATOR huntbouts_crosscat FOR bout_population;

""


In [11]:
%mml INITIALIZE 100 MODELS IF NOT EXISTS FOR huntbouts_crosscat;

""


In [ ]:
%mml ANALYZE huntbouts_crosscat FOR 100 ITERATIONS (OPTIMIZED);
# note you can use the (OPTIMIZED) flag here but that gave you weird results last time. 

Completed: 100 iterations in 18.103925 seconds.
Completed: 100 iterations in 18.854704 seconds.
Completed: 100 iterations in 16.048822 seconds.
Completed: 100 iterations in 17.320630 seconds.
Completed: 100 iterations in 16.722027 seconds.
Completed: 100 iterations in 21.690960 seconds.
Completed: 100 iterations in 20.494181 seconds.
Completed: 100 iterations in 17.901351 seconds.
Completed: 100 iterations in 17.334499 seconds.
Completed: 100 iterations in 26.912325 seconds.
Completed: 100 iterations in 30.061783 seconds.
Completed: 100 iterations in 26.093016 seconds.
Completed: 100 iterations in 19.184071 seconds.
Completed: 100 iterations in 16.881250 seconds.
Completed: 100 iterations in 29.642560 seconds.
Completed: 100 iterations in 27.553627 seconds.
Completed: 100 iterations in 21.163803 seconds.
Completed: 100 iterations in 29.810577 seconds.
Completed: 100 iterations in 22.614089 seconds.
Completed: 100 iterations in 21.466970 seconds.
Completed: 100 iterations in 20.425789 s